# TSP Genetic Algorithm — Analysis & Experiments

**Research-grade notebook**: Detailed explanation, implementation, experiments, and plots for the Travelling Salesman Problem solved using a Genetic Algorithm (GA). This notebook includes mathematical background, implementation, parameter tuning experiments, and visualizations of convergence.

---

## 1. Background

**Travelling Salesman Problem (TSP)**: Given `n` cities and pairwise distances, find the shortest possible route that visits each city exactly once and returns to the start.

- TSP is **NP-hard**: exact solutions are exponential in the worst case.
- **Genetic Algorithms (GAs)** are population-based metaheuristics that use selection, crossover, and mutation to evolve candidate solutions.

**Notebook goals**:
1. Implement a clear, well-documented GA for TSP.
2. Track best & average fitness across generations.
3. Visualize convergence and route.
4. Provide interpretation and parameter studies.

In [ ]:
import random
import math
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import time
import os

random.seed(42)
np.random.seed(42)

os.makedirs('output_notebook', exist_ok=True)
print('Setup complete')

In [ ]:
def distance(a, b):
    return math.hypot(a[0]-b[0], a[1]-b[1])

def total_distance(route, cities):
    dist = 0.0
    for i in range(len(route)):
        dist += distance(cities[route[i]], cities[route[(i+1) % len(route)]])
    return dist

def plot_route(route, cities, title='TSP Route', filename=None):
    xs = [cities[i][0] for i in route] + [cities[route[0]][0]]
    ys = [cities[i][1] for i in route] + [cities[route[0]][1]]
    plt.figure(figsize=(7,5))
    plt.plot(xs, ys, '-o')
    for i, idx in enumerate(route):
        plt.text(cities[idx][0], cities[idx][1], str(idx))
    plt.title(title)
    plt.xlabel('X'); plt.ylabel('Y')
    plt.grid(True)
    if filename:
        plt.savefig(filename, bbox_inches='tight')
    plt.show()

In [ ]:
def create_population(pop_size, n_cities):
    base = list(range(n_cities))
    pop = []
    for _ in range(pop_size):
        r = base[:]
        random.shuffle(r)
        pop.append(r)
    return pop

def tournament_selection(population, cities, k=3):
    candidates = random.sample(population, k)
    candidates.sort(key=lambda r: total_distance(r, cities))
    return candidates[0]

def ordered_crossover(p1, p2):
    n = len(p1)
    a, b = sorted(random.sample(range(n), 2))
    child = [None]*n
    child[a:b] = p1[a:b]
    ptr = 0
    for g in p2:
        if g not in child:
            while child[ptr] is not None:
                ptr += 1
            child[ptr] = g
    return child

def swap_mutation(route, mutation_rate=0.02):
    r = route[:]
    for i in range(len(r)):
        if random.random() < mutation_rate:
            j = random.randrange(len(r))
            r[i], r[j] = r[j], r[i]
    return r

def genetic_algorithm_experiment(cities, pop_size=200, generations=500, mutation_rate=0.02, k=3, elite_size=2, verbose=False):
    n = len(cities)
    population = create_population(pop_size, n)
    best_history = []
    avg_history = []
    best_route = None
    best_dist = float('inf')
    start_time = time.time()
    for gen in range(generations):
        distances = [total_distance(r, cities) for r in population]
        min_d = min(distances)
        avg_d = sum(distances)/len(distances)
        best_history.append(min_d)
        avg_history.append(avg_d)
        if min_d < best_dist:
            best_dist = min_d
            best_route = deepcopy(population[distances.index(min_d)])
        ranked = sorted(zip(population, distances), key=lambda x: x[1])
        new_pop = [deepcopy(ranked[i][0]) for i in range(elite_size)]
        while len(new_pop) < pop_size:
            p1 = tournament_selection(population, cities, k)
            p2 = tournament_selection(population, cities, k)
            child = ordered_crossover(p1, p2)
            child = swap_mutation(child, mutation_rate)
            new_pop.append(child)
        population = new_pop
        if verbose and (gen % max(1, generations//10) == 0):
            print(f"Gen {gen:4d} | best {best_dist:.4f} | avg {avg_d:.4f}")
    elapsed = time.time() - start_time
    return {'best_route': best_route, 'best_distance': best_dist, 'best_history': best_history, 'avg_history': avg_history, 'time': elapsed}

In [ ]:
def random_cities(n=20, xrange=(0,100), yrange=(0,100)):
    return {i: (random.uniform(*xrange), random.uniform(*yrange)) for i in range(n)}

cities = random_cities(20)
print('Created', len(cities), 'cities')

In [ ]:
res = genetic_algorithm_experiment(cities, pop_size=300, generations=400, mutation_rate=0.02, k=5, elite_size=4, verbose=True)
print(f"Best distance: {res['best_distance']:.4f} | time: {res['time']:.2f}s")

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,5))
plt.plot(res['best_history'], label='Best')
plt.plot(res['avg_history'], label='Average')
plt.xlabel('Generation')
plt.ylabel('Distance')
plt.title('GA Convergence')
plt.legend()
plt.grid(True)
plt.savefig('output_notebook/fitness_curve.png', bbox_inches='tight')
plt.show()

In [ ]:
plot_route(res['best_route'], cities, title=f"Best route (distance={res['best_distance']:.2f})", filename='output_notebook/best_route_notebook.png')

In [ ]:
import pandas as pd
# small parameter sweep
sweep = []
for pop in [100,200]:
    for mut in [0.01,0.02]:
        for elite in [0,2]:
            print('Running', pop, mut, elite)
            r = genetic_algorithm_experiment(cities, pop_size=pop, generations=200, mutation_rate=mut, k=5, elite_size=elite)
            sweep.append({'pop':pop,'mut':mut,'elite':elite,'best':r['best_distance'],'time':r['time']})
df = pd.DataFrame(sweep).sort_values('best')
df.to_csv('output_notebook/parameter_sweep.csv', index=False)
df

## Conclusions

- GA reliably reduces distance; tuning population, mutation, and elitism impacts final quality and runtime.
- Use larger population for harder instances. Use mutation to maintain diversity.

---

Files saved to `output_notebook/`:
- `fitness_curve.png`
- `best_route_notebook.png`
- `parameter_sweep.csv`

You can download this notebook using the link provided.